# Part 3, Topic 3: DPA on Firmware Implementation of AES (HARDWARE)

---
**THIS IS NOT THE COMPLETE TUTORIAL - see file with `(MAIN)` in the name.**

---

First you'll need to select which hardware setup you have. You'll need to select a `SCOPETYPE`, a `PLATFORM`, and a `CRYPTO_TARGET`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. `CRYPTO_TARGET` selects the crypto implementation, with `'TINYAES128C'` working on all platforms. An alternative for `'CWLITEXMEGA'` targets is `'AVRCRYPTOLIB'`. For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'
```

In [12]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='NONE'
SS_VER='SS_VER_1_1'

The following code will build the firmware for the target.

In [14]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [15]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

Building for platform CWLITEARM with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
make clean_objs .dep 
make[1]: Entering directory '/home/daniel/Documents/Uni/Semester 2/Crypt/Assignment2/ChipWhisperer/hardware/victims/firmware/simpleserial-aes'
Building for platform CWLITEARM with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
rm -f -- simpleserial-aes-CWLITEARM.hex
rm -f -- simpleserial-aes-CWLITEARM.eep
rm -f -- simpleserial-aes-CWLITEARM.cof
rm -f -- simpleserial-aes-CWLITEARM.elf
rm -f -- simpleserial-aes-CWLITEARM.map
rm -f -- simpleserial-aes-CWLITEARM.sym
rm -f -- simpleserial-aes-CWLITEARM.lss
rm -f -- objdir-CWLITEARM/*.o
rm -f -- objdir-CWLITEARM/*.lst
rm -f -- simpleserial-aes.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-aes.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-aes.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
make[1]: '.dep' is up to date.
make[1]: 

/usr/lib/gcc/arm-none-eabi/11.2.0/../../../../arm-none-eabi/bin/ld: objdir-CWLITEARM/simpleserial-aes.o: in function `get_mask':
/home/daniel/Documents/Uni/Semester 2/Crypt/Assignment2/ChipWhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c:27: undefined reference to `aes_indep_mask'
/usr/lib/gcc/arm-none-eabi/11.2.0/../../../../arm-none-eabi/bin/ld: objdir-CWLITEARM/simpleserial-aes.o: in function `get_key':
/home/daniel/Documents/Uni/Semester 2/Crypt/Assignment2/ChipWhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c:33: undefined reference to `aes_indep_key'
/usr/lib/gcc/arm-none-eabi/11.2.0/../../../../arm-none-eabi/bin/ld: objdir-CWLITEARM/simpleserial-aes.o: in function `get_pt':
/home/daniel/Documents/Uni/Semester 2/Crypt/Assignment2/ChipWhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c:39: undefined reference to `aes_indep_enc_pretrigger'
/usr/lib/gcc/arm-none-eabi/11.2.0/../../../../arm-none-eabi/bin/ld: /home/danie

make[1]: Leaving directory '/home/daniel/Documents/Uni/Semester 2/Crypt/Assignment2/ChipWhisperer/hardware/victims/firmware/simpleserial-aes'


CalledProcessError: Command 'b'cd ../../../hardware/victims/firmware/simpleserial-aes\nmake PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3\n'' returned non-zero exit status 2.

In [4]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

NameError: name 'scope' is not defined

For this tutorial, we'll need to capture around 2500 traces:

In [6]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 2500
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

/tmp/ipykernel_20586/543036540.py:14: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(N, desc='Capturing traces'):


Capturing traces:   0%|          | 0/2500 [00:00<?, ?it/s]


KeyboardInterrupt



In [9]:
scope.dis()

ERROR:ChipWhisperer Scope:Scope already disconnected!


True